In [1]:
import pandas as pd
import numpy as np

# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

This notebooks loads and cleans up the original data, preparing a data set for the analysis of the probability of completing (redeeming) the promotional offers. First, we do some initial cleaning on the three data sets loaded from json files. We also drop the 2175 users for which there are no data on gender, income, and age (for those users, the age is set to 118 as a placeholder), as we won't be able to use these observations to examine how the probability of completing an offer differs on sociodemographic characteristics.

In [2]:
portfolio = portfolio.rename(columns={'id': 'offer_id', 
                                      'duration': 'offer_duration',
                                      'reward': 'offer_reward'})
portfolio['offer_duration_hours'] = portfolio['offer_duration']*24

# drop users with missing gender and income
print('Original number of users: ' + str(profile.shape[0])) 
profile_cleaned = profile.dropna(subset=['gender', 'income']).rename(
    columns={'id': 'user_id'})
print('The number of users after dropping missing observations: ' + str(profile_cleaned.shape[0])) 

# clarify the gender variable
gender_sub = {'F': 'Female', 'M': 'Male', 'O': 'Other'}
profile_cleaned['gender'] = profile_cleaned['gender'].apply(
    lambda x: ' '.join([gender_sub.get(i, i) for i in x.split()]))

Original number of users: 17000
The number of users after dropping missing observations: 14825


Then, in the data set of transactions, we expand the column *value*, which is a dictionary that includes either the amount of a given transaction or the offer id if the transaction is receiving, viewing, or completing the offer. (We won't need transaction amounts for the analysis of the probability of completing offers.)

In [3]:
transcript = pd.concat([transcript, 
                        pd.DataFrame.from_records(transcript.value.tolist())],
                        axis=1)
transcript['offer_id'].fillna(transcript['offer id'], inplace=True)
transcript = transcript.drop(columns=['offer id', 'value'])

# drop records for users without demographic data
transcript_u = transcript[
    transcript.person.isin(profile_cleaned.user_id)
    ].sort_values(by=['person', 'time']).rename(columns={'person': 'user_id'})

Then, we create a new data frame that captures information on each offer each user received. Each row in this data frame includes information on a particular offer received by a particular user: when the offer was received, and whether and when it was viewed or completed. To create the data, we loop through all users in the transaction data set, and for each user, we reconstruct what happened with respect to a particular offer by looking at all the transactions with the corresponding offer id in the period when the offer was active.

One caveat about this recovery process is that in some cases, users were sent the same offer two or more weeks in a row. Given that offer duration is sometimes longer than a week, in some of these cases, two offers with the same id were active simultaneously for a few days, and if offer viewing or completion happened in that specific period, there is no way to determine which of these two identical offers was viewed or completed because the same offer sent multiple times always has the same id. However, this issue is not severe, as in the overwhelming majority of cases, offers were viewed or completed soon after they were sent (or not viewed/completed at all), as the histograms below will show. A manual inspection of the original transactions data set with respect to specific users who had multiple identical offers in a row has also shown that this problem is very rare. 

In [4]:
# three functions to create a user-offer data frame
def get_offer_data(user_data, offer, time_received, current_offer_duration):
    
    '''
    Extract data on a given offer received by a particular user:
    whether and when the offer was viewed and/or completed
    
    Args:
        user_data: a subset of transactions for the specific user
        offer: specific offer id
        time_received: time at which the offer was sent
        current_offer_duration: the duration of the specific offer in hours
        
    Returns: data frame with information on the reception, viewing,
    and completion of a particular offer by a particular user
    '''
    
    # limit transactions to the period in which the offer was active
    offer_data = user_data[
        (user_data.time >= time_received) &
        (user_data.time <= time_received + current_offer_duration) &
        (user_data.offer_id == offer)]
    
    # check whether the offer was completed/redeemed in that period
    offer_completed = int('offer completed' in offer_data.event.tolist())
    
    # check whether the offer was viewed in that period
    offer_viewed = int('offer viewed' in offer_data.event.tolist())
    
    # capture when the offer was completed/viewed
    if offer_completed == 1:
        time_completed = offer_data[
            offer_data.event == 'offer completed'].time.iloc[0]
    else:
        time_completed = np.nan
        
    if offer_viewed == 1:
        time_viewed = offer_data[
            offer_data.event == 'offer viewed'].time.iloc[0]
    else:
        time_viewed = np.nan
        
    # check whether the offer was completed before viewing it
    if offer_completed == 1 & offer_viewed == 1:
        if time_completed < time_viewed:
            offer_viewed_before = 0
        else:
            offer_viewed_before = 1
    elif offer_completed == 1 & offer_viewed == 0:
        offer_viewed_before = 0
    # set to nan if offer not completed
    else:
        offer_viewed_before = np.nan
    
    # combine everything into a data frame            
    user_offer_data = pd.DataFrame(
        {'user_id': offer_data.user_id.unique().tolist(),
         'offer_id': [offer],
         'completed': [offer_completed],
         'viewed': [offer_viewed],
         'viewed_before': [offer_viewed_before],
         'time_received': [time_received],
         'time_viewed': [time_viewed],
         'time_completed': [time_completed],
         'offer_duration': [current_offer_duration]})
    
    return user_offer_data

def check_user_offers(user_data, offer):
    
    '''
    Check how many times a given offer was received by a particular user
    and extract data on offer viewing/completion
    
    Args:
        user_data: a subset of transactions for the specific user
        offer: specific offer id
        
    Returns: data frame with information on the reception, viewing,
    and completion of all offers with a particular id by a particular user
    '''
    
    # extract the duration of the offer in hours from offer metadata
    current_offer_duration = portfolio[
        portfolio.offer_id == offer].offer_duration_hours.item()
    
    # check how many times the user received the same offer
    received_times = user_data[(user_data.event == 'offer received') &
                               (user_data.offer_id == offer)].time.tolist()
    
    # extract offer data
    user_offer_data = [get_offer_data(user_data, offer,
                                      time_point,
                                      current_offer_duration) for
                       time_point in received_times]
    user_offer_data = pd.concat(user_offer_data)
    
    # how many times the same offer was received 
    user_offer_data['offer_count'] = len(received_times)
    user_offer_data['time_points'] = '.'.join(str(x) for x in received_times)
        
    return user_offer_data
        

def clean_offers_by_user(user):
    
    '''
    Get data on all offers received by a particular user
    
    Args:
        user: user id
        
    Returns: data frame with information on the reception, viewing,
    and completion of all offers received by a particular user
    '''
    
    # a data set of all records on offers received, viewed, or completed
    # by a particular user
    user_transactions = transcript_u[transcript_u.user_id == user]
    
    # the ids of all offers received by the user
    user_offers = user_transactions.offer_id.unique().tolist()
    
    # extract data on all offers received by this user
    user_offer_data = [check_user_offers(user_transactions, offer) for 
                       offer in user_offers]
    user_offer_data = pd.concat(user_offer_data)
    
    return user_offer_data